In [ ]:
# -*- coding: utf-8 -*-
"""
@author:  Zhang 
@date: 2018-06-26
"""

import datetime
import ipywidgets as widgets
import IPython.display
import pymongo
import pandas
import shlex

# Database indicator for date storage after flattening
DatePathStr = 'DataPath'

class View:
    def __init__(self):
        self.date_start = widgets.DatePicker(
            description='Start Date',
            value=datetime.date(2015, 9, 7),
            disabled=False)

        self.date_end = widgets.DatePicker(
            description='End Date',
            value=datetime.date.today(),
            disabled=False)

        self.date = widgets.HBox([self.date_start, self.date_end])

        self.key = widgets.Text(
            value='',
            placeholder='Enter search query',
            description='',
            disabled=False,
            layout=widgets.Layout(width='61.5%',heigh='80px'))

        self.button = widgets.Button(description="Do search")
        
        self.refresh_console()

    def refresh_console(self):
        IPython.display.clear_output()

        IPython.display.display(self.date)
        IPython.display.display(self.key)
        IPython.display.display(self.button)

    def show_results(self, results):
        print("{} items found.".format(len(results)))

        # Display results
        max_items_displayed = 100
        if len(results) < max_items_displayed:
            for result in results:
                raw_data = result.pop('Rohdaten', None)
                df = pandas.DataFrame({'values':pandas.Series(result)})
                display(df)
                df = df = pandas.DataFrame({'values':pandas.Series(raw_data)})
                display(df)
        else:
            print("More than {} items found. Narrow your search criteria.".format(max_items_displayed))


class Model:
    collection = None

    def __init__(self):
        self.connect_db()

    def connect_db(self):
        client = pymongo.MongoClient()
        client = pymongo.MongoClient('localhost', 27017)
        db = client.mdb
        #mdb is the database created
        print("Connected to the database.")
        self.collection = db.measure2

        # Display amount of database items
#         cursor = self.collection.find()
#         print("The database contains", cursor.count(), "measurements.")

    def search_recursive(self, date_start, date_end, key):
        ss = date_start.strftime('%Y-%m-%d')
        en = date_end.strftime('%Y-%m-%d')

        # Split search field into individual search terms
        
        input_str=shlex.shlex(key,posix=True)
        input_str.whitespace=' '
        input_str.whitesapce_split=True
        search_keys=list(input_str)
        print(search_keys)

        # Create iterator for all database entries
        # in the selected time range
        cursor = self.collection.find({
            'DataPath': {
                '$gte': ss,
                '$lte': en
            }
        })

        # Do a full text search in the found entries
        found_items = []
        if cursor.count() > 0:
            # Iterate over all database entries
            for item in cursor:
                match = True
                # Check if all search_keys are found in the item
                for search_key in search_keys:
                    # Make all search queries case insensitive
                    if search_key.lower() in str(item).lower():
                        match = False
                        break
                # All search keys have been found
                if match == True:
                    found_items.append(item)

        return found_items


class Controller:
    def __init__(self):
        # Add model and view
        self.view = View()
        self.model = Model()

        # Register events
        self.view.button.on_click(self.do_search)

    def do_search(self, b):
        self.view.refresh_console()
        results = self.model.search_recursive(self.view.date_start.value,
                                    self.view.date_end.value,
                                    self.view.key.value)
        self.view.show_results(results)


def main():
    c = Controller()


if __name__ == "__main__":
    main()